# Dataset

In [1]:
#To ignore warnings.
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Load the dataset.
import pandas as pd
df=pd.read_csv("Dataset/spam.csv", encoding='latin-1')
#Remove unwanted columns.
df = df.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
#Name the columns.
df.columns = ["label", "text"]
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# Data preprocessing

In [3]:
import nltk,string

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/dev

In [4]:
#Getting the english stopwords.
stopwords = nltk.corpus.stopwords.words('english')

In [5]:
#Function to clean the text.
def clean_text(text):
    #remove punctuation
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    #separate into tokens
    tokens = text.split()
    #remove stopwords
    text = [word for word in tokens if word not in stopwords]
    return text


In [6]:
#Apply the clean_text function 
df["cleaned_text"]=df["text"].apply(clean_text)

In [7]:
df.head()

,label,text,cleaned_text
0,ham,"Go until jurong point, crazy.. Available only ...","[go, jurong, point, crazy, available, bugis, n..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,U dun say so early hor... U c already then say...,"[u, dun, say, early, hor, u, c, already, say]"
4,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, dont, think, goes, usf, lives, around, t..."


# doc2vec

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
#Split the data (train, test).
X_train, X_test, y_train, y_test = train_test_split(df["cleaned_text"],df['label'],test_size=0.2)

In [10]:
import gensim
print("Gensim version:",gensim.__version__)
import numpy as np

Gensim version: 4.0.1


In [11]:
#Creating Tagged document vector for train datpoints.
tagDoc_train=[gensim.models.doc2vec.TaggedDocument(v,[i]) for i,v in enumerate (X_train)]
#Creating Tagged document vector for test datpoints.
tagDoc_test=[gensim.models.doc2vec.TaggedDocument(v,[i]) for i,v in enumerate (X_test)]

In [12]:
tagDoc_train[:10]

[TaggedDocument(words=['going', 'join', 'tomorrow'], tags=[0]),
 TaggedDocument(words=['fuck', 'babe', 'miss', 'sooooo', 'much', 'wish', 'sleep', 'bed', 'lonely', 'go', 'sleep', 'dream', 'love'], tags=[1]),
 TaggedDocument(words=['pls', 'give', 'food', 'preferably', 'pap', 'slowly', 'loads', 'sugar', 'take', 'hour', 'give', 'water', 'slowly'], tags=[2]),
 TaggedDocument(words=['kkapo', 'kgood', 'movie'], tags=[3]),
 TaggedDocument(words=['yar', 'quite', 'clever', 'aft', 'many', 'guesses', 'lor', 'got', 'ask', '2', 'bring', 'thk', 'darren', 'willing', '2', 'go', 'aiya', 'thk', 'leona', 'still', 'attach', 'wat'], tags=[4]),
 TaggedDocument(words=['call', 'like', 'ltgt', 'times', 'oh', 'give', 'us', 'hypertension', 'oh'], tags=[5]),
 TaggedDocument(words=['sorry', 'uncle', 'ill', 'keep', 'touch'], tags=[6]),
 TaggedDocument(words=['tiime', 'dont', 'let', 'hug', 'dont', 'break', 'tears'], tags=[7]),
 TaggedDocument(words=['yeah', 'totes', 'u', 'wanna'], tags=[8]),
 TaggedDocument(words=['g

In [13]:
#Train doc2vec model.
d2v_model=gensim.models.Doc2Vec(tagDoc_train,vector_size=100,window=5,min_count=2)

In [14]:
#Covert text into learned word vector.
train_vec=[d2v_model.infer_vector(td.words) for td in tagDoc_train]
test_vec=[d2v_model.infer_vector(td.words) for td in tagDoc_test]

# Lets build the model !

In [15]:
from sklearn.ensemble import RandomForestClassifier

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\hp\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ._gradient_boosting import predict_stages
C:\Users\hp\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modif

In [16]:
#Build the Random Forest Classifier.
rf=RandomForestClassifier()
#Fit the model with training data.
rf_model=rf.fit(train_vec,y_train)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:482: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_store_unique_indices = np.zeros(y.shape, dtype=np.int)
C:\Users\hp\Anaconda3\lib\site-packages\sklearn\tree\tree.py:149: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current us

In [17]:
#Predict the output for testing data.
y_pred=rf_model.predict(test_vec)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:157: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


# Evaluation

In [18]:
#Calculate precision score and recall score.
from sklearn.metrics import precision_score, recall_score,accuracy_score
#Precision - ability of the classifier to find all the positive samples.
precision = precision_score(y_test, y_pred, pos_label='spam')
#Recall - ability of the classifier to find true positive samples.
recall = recall_score(y_test, y_pred, pos_label='spam')
print("Precision score: ",precision)
print("Recall score: ",recall)


Precision score:  0.9821428571428571
Recall score:  0.6832298136645962


In [19]:
import numpy as np
y_pred=np.where(y_pred == "spam", 1, 0)

In [20]:
y_test=np.where(np.array(y_test)== "spam", 1, 0)

In [21]:
#Test datapoints accuracy.
accuracy=accuracy_score(y_test, y_pred)
print("Test Accuracy: ",accuracy)


Test Accuracy:  0.9524663677130045
